# Week 9, Day 1: Introduction to Generative AI

## Learning Objectives
- Understand generative AI concepts
- Learn different types of generative models
- Master basic generation techniques
- Practice implementing generators

## Topics Covered
1. Generative Models Overview
2. Variational Autoencoders (VAEs)
3. Generative Adversarial Networks (GANs)
4. Diffusion Models

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

## 1. Variational Autoencoder (VAE)

In [ ]:
class VAE(tf.keras.Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(28, 28, 1)),
            layers.Conv2D(32, 3, activation='relu', strides=2, padding='same'),
            layers.Conv2D(64, 3, activation='relu', strides=2, padding='same'),
            layers.Flatten(),
            layers.Dense(latent_dim + latent_dim)
        ])
        
        # Decoder
        self.decoder = tf.keras.Sequential([
            layers.Input(shape=(latent_dim,)),
            layers.Dense(7*7*32, activation='relu'),
            layers.Reshape((7, 7, 32)),
            layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same'),
            layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same'),
            layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')
        ])
    
    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar
    
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    
    def decode(self, z):
        return self.decoder(z)
    
    def call(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        x_logit = self.decode(z)
        return x_logit, mean, logvar

## 2. Generative Adversarial Network (GAN)

In [ ]:
class GAN:
    def __init__(self, latent_dim):
        self.latent_dim = latent_dim
        
        # Generator
        self.generator = tf.keras.Sequential([
            layers.Input(shape=(latent_dim,)),
            layers.Dense(7*7*256, use_bias=False),
            layers.BatchNormalization(),
            layers.LeakyReLU(),
            layers.Reshape((7, 7, 256)),
            layers.Conv2DTranspose(128, 5, strides=1, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.LeakyReLU(),
            layers.Conv2DTranspose(64, 5, strides=2, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.LeakyReLU(),
            layers.Conv2DTranspose(1, 5, strides=2, padding='same', use_bias=False,
                                  activation='tanh')
        ])
        
        # Discriminator
        self.discriminator = tf.keras.Sequential([
            layers.Input(shape=(28, 28, 1)),
            layers.Conv2D(64, 5, strides=2, padding='same'),
            layers.LeakyReLU(),
            layers.Dropout(0.3),
            layers.Conv2D(128, 5, strides=2, padding='same'),
            layers.LeakyReLU(),
            layers.Dropout(0.3),
            layers.Flatten(),
            layers.Dense(1)
        ])
        
        # Optimizers
        self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
    
    def generator_loss(self, fake_output):
        return tf.keras.losses.BinaryCrossentropy(
            from_logits=True
        )(tf.ones_like(fake_output), fake_output)
    
    def discriminator_loss(self, real_output, fake_output):
        real_loss = tf.keras.losses.BinaryCrossentropy(
            from_logits=True
        )(tf.ones_like(real_output), real_output)
        fake_loss = tf.keras.losses.BinaryCrossentropy(
            from_logits=True
        )(tf.zeros_like(fake_output), fake_output)
        return real_loss + fake_loss

## 3. Diffusion Model

In [ ]:
class DiffusionModel:
    def __init__(self, timesteps=1000):
        self.timesteps = timesteps
        self.beta = np.linspace(0.0001, 0.02, timesteps)
        self.alpha = 1. - self.beta
        self.alpha_bar = np.cumprod(self.alpha)
        
        # Noise predictor network
        self.model = tf.keras.Sequential([
            layers.Input(shape=(28, 28, 1)),
            layers.Conv2D(64, 3, activation='relu', padding='same'),
            layers.Conv2D(128, 3, activation='relu', padding='same'),
            layers.Conv2D(256, 3, activation='relu', padding='same'),
            layers.Conv2D(128, 3, activation='relu', padding='same'),
            layers.Conv2D(64, 3, activation='relu', padding='same'),
            layers.Conv2D(1, 3, padding='same')
        ])
    
    def forward_diffusion(self, x0, t):
        noise = tf.random.normal(shape=x0.shape)
        alpha_t = tf.gather(self.alpha_bar, t)
        alpha_t = tf.reshape(alpha_t, (-1, 1, 1, 1))
        return tf.sqrt(alpha_t) * x0 + tf.sqrt(1. - alpha_t) * noise, noise
    
    def reverse_diffusion(self, x, t):
        predicted_noise = self.model(x, training=False)
        alpha_t = tf.gather(self.alpha_bar, t)
        alpha_t = tf.reshape(alpha_t, (-1, 1, 1, 1))
        beta_t = tf.gather(self.beta, t)
        beta_t = tf.reshape(beta_t, (-1, 1, 1, 1))
        
        mean = (1. / tf.sqrt(alpha_t)) * (x - ((1. - alpha_t) / tf.sqrt(1. - alpha_t)) * predicted_noise)
        variance = beta_t
        
        return mean + tf.sqrt(variance) * tf.random.normal(shape=x.shape)

## Practical Exercises

In [ ]:
# Exercise 1: Simple VAE

def vae_exercise():
    print("Task: Implement a basic VAE")
    print("1. Create encoder")
    print("2. Create decoder")
    print("3. Train model")
    print("4. Generate samples")
    
    # Your code here

vae_exercise()

In [ ]:
# Exercise 2: Simple GAN

def gan_exercise():
    print("Task: Implement a basic GAN")
    print("1. Create generator")
    print("2. Create discriminator")
    print("3. Train adversarial model")
    print("4. Generate samples")
    
    # Your code here

gan_exercise()

## MCQ Quiz

1. What is generative AI?
   - a) Classification model
   - b) Content creation model
   - c) Regression model
   - d) Clustering model

2. What is a VAE?
   - a) Classification model
   - b) Generative model
   - c) Regression model
   - d) Clustering model

3. What is a GAN?
   - a) Single network
   - b) Adversarial networks
   - c) Classification model
   - d) Regression model

4. What is a diffusion model?
   - a) Classification model
   - b) Noise-based generation
   - c) Regression model
   - d) Clustering model

5. What is the latent space?
   - a) Input space
   - b) Compressed representation
   - c) Output space
   - d) Feature space

6. What is the role of the discriminator?
   - a) Generate samples
   - b) Detect fake samples
   - c) Compress data
   - d) Process input

7. What is the purpose of reparameterization?
   - a) Data processing
   - b) Backpropagation trick
   - c) Model evaluation
   - d) Data augmentation

8. What is mode collapse?
   - a) Model success
   - b) GAN problem
   - c) Training method
   - d) Evaluation metric

9. What is the ELBO loss?
   - a) Classification loss
   - b) VAE objective
   - c) GAN loss
   - d) Regression loss

10. What is denoising diffusion?
    - a) Data cleaning
    - b) Generation process
    - c) Model architecture
    - d) Loss function

Answers: 1-b, 2-b, 3-b, 4-b, 5-b, 6-b, 7-b, 8-b, 9-b, 10-b